In [ ]:
    # CS4387 Assignment2 CNNs - Created by Kevin Hough (21307571), Thomas Mccarthy (21327696) and Eamonn Gorman (21204616)

In [ ]:
#FIRST EDITION

In [1]:
# Loading data 
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ikjotsingh221/animal-dataset")

print("Path to dataset files:", path)

C:\Users\Kevin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Kevin\.cache\kagglehub\datasets\ikjotsingh221\animal-dataset\versions\1


In [2]:
import sys
assert sys.version_info >= (3, 5)

import tensorflow as tf
from tensorflow.keras import layers
assert tf.__version__ >= "2.0"

import sklearn
assert sklearn.__version__ >= "0.20"

IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.") 
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")
        
import numpy as np
import os 

np.random.seed(54)
tf.random.set_seed(54)

dataset_path = path # Replace with the actual path


batch_size = 32
img_size = (299, 299)

#Load the dataset from the directory
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

#Get class names from the dataset
class_names = train_dataset.class_names
print("Classes:", class_names)

No GPU was detected. CNNs can be very slow without a GPU.
Found 1564 files belonging to 1 classes.
Using 1252 files for training.
Found 1564 files belonging to 1 classes.
Using 312 files for validation.
Classes: ['dataset']


In [3]:
def preprocess_image(image, label):
    """
    Normalizing image Data.
    
    This uses the function by tensorflow to preprocess the image since Inception_V3 needs pixell values between 1 to -1.
    
    Goes through and process each of the images and returns the processed image and their label
    """
    image = tf.keras.applications.inception_v3.preprocess_input(image)
    return image, label


train_dataset = train_dataset.map(preprocess_image)
validation_dataset = validation_dataset.map(preprocess_image)

# Augment data to prevent overfitting of the model
data_augmentation = tf.keras.Sequential({
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.5),
    layers.RandomZoom(0.3)
})

# Apply augmentation to training set
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x), y))


In [10]:
base_model = tf.keras.applications.InceptionV3(
    input_shape=(299, 299, 3),
    include_top=False,  
    weights="imagenet"  
)

#Freeze base model
base_model.trainable = False 

model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(len(class_names), activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 16s 0us/step


In [11]:
# Summarize the model architecture
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)       │ (None, 8, 8, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         2,049 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,804,833 (83.18 MB)

 Trainable params: 2,049 (8.00 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [12]:
history= model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10
)

base_model.trainable= True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history_fine = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=5  # Fine-tune for additional epochs
)

model.save("inception_v3_fine_tuned_model")

Epoch 1/10


KeyboardInterrupt: 